<a href="https://colab.research.google.com/github/sudeshna0501/twitter-sentiment-analysis/blob/main/Multichannel_CNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline

In [ ]:
from google.colab import drive
drive.mount("/content/gdrive")


Mounted at /content/gdrive


In [ ]:
import pandas as pd
df=pd.read_csv('/content/gdrive/My Drive/Tweets.csv')

In [ ]:
X= df['text']
Y= df['airline_sentiment']

In [ ]:
from sklearn.preprocessing import LabelEncoder
from sklearn.pipeline import Pipeline

In [ ]:
Y

0         neutral
1        positive
2         neutral
3        negative
4        negative
           ...   
14635    positive
14636    negative
14637     neutral
14638    negative
14639     neutral
Name: airline_sentiment, Length: 14640, dtype: object

In [ ]:
encoder = LabelEncoder()
encoder.fit(Y)

LabelEncoder()

In [ ]:
from keras.utils import np_utils

In [ ]:

encoded_Y = encoder.transform(df['airline_sentiment'])
# convert integers to dummy variables (i.e. one hot encoded)
dummy_y = np_utils.to_categorical(encoded_Y)

In [ ]:
dummy_y

array([[0., 1., 0.],
       [0., 0., 1.],
       [0., 1., 0.],
       ...,
       [0., 1., 0.],
       [1., 0., 0.],
       [0., 1., 0.]], dtype=float32)

In [ ]:
df.head()

,tweet_id,airline_sentiment,airline_sentiment_confidence,negativereason,negativereason_confidence,airline,airline_sentiment_gold,name,negativereason_gold,retweet_count,text,tweet_coord,tweet_created,tweet_location,user_timezone
0,570306133677760513,neutral,1.0000,NaN,NaN,Virgin America,NaN,cairdin,NaN,0,@VirginAmerica What @dhepburn said.,NaN,2015-02-24 11:35:52 -0800,NaN,Eastern Time (US & Canada)
1,570301130888122368,positive,0.3486,NaN,0.0000,Virgin America,NaN,jnardino,NaN,0,@VirginAmerica plus you've added commercials t...,NaN,2015-02-24 11:15:59 -0800,NaN,Pacific Time (US & Canada)
2,570301083672813571,neutral,0.6837,NaN,NaN,Virgin America,NaN,yvonnalynn,NaN,0,@VirginAmerica I didn't today... Must mean I n...,NaN,2015-02-24 11:15:48 -0800,Lets Play,Central Time (US & Canada)
3,570301031407624196,negative,1.0000,Bad Flight,0.7033,Virgin America,NaN,jnardino,NaN,0,@VirginAmerica it's really aggressive to blast...,NaN,2015-02-24 11:15:36 -0800,NaN,Pacific Time (US & Canada)
4,570300817074462722,negative,1.0000,Can't Tell,1.0000,Virgin America,NaN,jnardino,NaN,0,@VirginAmerica and it's a really big bad thing...,NaN,2015-02-24 11:14:45 -0800,NaN,Pacific Time (US & Canada)


In [ ]:
import nltk

In [ ]:
import string
from nltk.corpus import stopwords
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
# turn a doc into clean tokens
def clean_doc(doc):
	# split into tokens by white space
	tokens = doc.split()
	# remove punctuation from each token
	table = str.maketrans('', '', string.punctuation)
	tokens = [w.translate(table) for w in tokens]
	# remove remaining tokens that are not alphabetic
	tokens = [word for word in tokens if word.isalpha()]
	# filter out stop words
	stop_words = set(stopwords.words('english'))
	tokens = [w for w in tokens if not w in stop_words]
	# filter out short tokens
	tokens = [word for word in tokens if len(word) > 1]
	return tokens

In [ ]:
X = X.apply(clean_doc)

In [ ]:
X

0                    [VirginAmerica, What, dhepburn, said]
1        [VirginAmerica, plus, youve, added, commercial...
2        [VirginAmerica, didnt, today, Must, mean, need...
3        [VirginAmerica, really, aggressive, blast, obn...
4                 [VirginAmerica, really, big, bad, thing]
                               ...                        
14635    [AmericanAir, thank, got, different, flight, C...
14636    [AmericanAir, leaving, minutes, Late, Flight, ...
14637     [AmericanAir, Please, bring, American, Airlines]
14638    [AmericanAir, money, change, flight, dont, ans...
14639    [AmericanAir, ppl, need, know, many, seats, ne...
Name: text, Length: 14640, dtype: object

In [ ]:
from sklearn.model_selection import train_test_split


In [ ]:
X_train, X_test, y_train, y_test = \
train_test_split(X, dummy_y,test_size=0.2)

In [ ]:
y_test.shape

(2928, 3)

In [ ]:
from pickle import load
from numpy import array
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils.vis_utils import plot_model
from keras.models import Model
from keras.layers import Input
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import Dropout
from keras.layers import Embedding
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D
from keras.layers.merge import concatenate






# fit a tokenizer
def create_tokenizer(lines):
 tokenizer = Tokenizer()
 tokenizer.fit_on_texts(lines)
 return tokenizer


# calculate the maximum document length
def max_length(lines):
 return max([len(s.split()) for s in lines])


# encode a list of lines
def encode_text(tokenizer, lines, length):
 # integer encode
 encoded = tokenizer.texts_to_sequences(lines)
 # pad encoded sequences
 padded = pad_sequences(encoded, maxlen=length, padding='post')
 return padded




#for calculating length
def joining(ele):
 return " ".join(str(v) for v in ele)


# define the model
def define_model(length, vocab_size):
 # channel 1
 inputs1 = Input(shape=(length,))
 embedding1 = Embedding(vocab_size, 100)(inputs1)
 conv1 = Conv1D(filters=32, kernel_size=1, activation='relu')(embedding1)
 drop1 = Dropout(0.5)(conv1)
 pool1 = MaxPooling1D(pool_size=2)(drop1)
 flat1 = Flatten()(pool1)
 # channel 2
 inputs2 = Input(shape=(length,))
 embedding2 = Embedding(vocab_size, 100)(inputs2)
 conv2 = Conv1D(filters=32, kernel_size=2, activation='relu')(embedding2)
 drop2 = Dropout(0.5)(conv2)
 pool2 = MaxPooling1D(pool_size=2)(drop2)
 flat2 = Flatten()(pool2)
 # channel 3
 inputs3 = Input(shape=(length,))
 embedding3 = Embedding(vocab_size, 100)(inputs3)
 conv3 = Conv1D(filters=32, kernel_size=3, activation='relu')(embedding3)
 drop3 = Dropout(0.5)(conv3)
 pool3 = MaxPooling1D(pool_size=2)(drop3)
 flat3 = Flatten()(pool3)
 # merge
 merged = concatenate([flat1, flat2, flat3])
 # interpretation
 dense1 = Dense(4000, activation='relu')(merged)
 dense2 = Dense(2000, activation='relu')(dense1)
 dense3 = Dense(1000, activation='relu')(dense2)
 dense4 = Dense(500, activation='relu')(dense3)
 dense5 = Dense(200, activation='relu')(dense4)
 dense6 = Dense(100, activation='relu')(dense5)
 dense7 = Dense(20, activation='relu')(dense6)
 outputs = Dense(3, activation='softmax')(dense7)
 model = Model(inputs=[inputs1, inputs2, inputs3], outputs=outputs)
 # compile
 model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
 # summarize
 print(model.summary())
 plot_model(model, show_shapes=True, to_file='multichannel.png')
 return model


# create tokenizer
tokenizer = create_tokenizer(X_train)
# calculate max document length
joinned_train =X_train.apply(joining)
length = max_length(joinned_train)
# calculate vocabulary size
vocab_size = len(tokenizer.word_index) + 1
print('Max document length: %d' % length)
print('Vocabulary size: %d' % vocab_size)
# encode data
trainX = encode_text(tokenizer, X_train, length)
print(trainX.shape)


# define model
model = define_model(length, vocab_size)
print(model.summary())

# fit model
model.fit([trainX,trainX,trainX], y_train, epochs=10, batch_size=20)
# save the model
model.save('model.h5')

Max document length: 23
Vocabulary size: 11562
(11712, 23)
Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 23)]         0           []                               
                                                                                                  
 input_2 (InputLayer)           [(None, 23)]         0           []                               
                                                                                                  
 input_3 (InputLayer)           [(None, 23)]         0           []                               
                                                                                                  
 embedding (Embedding)          (None, 23, 100)      1156200     ['input_1[0][0]']                
                                   

In [ ]:
from pickle import load
from numpy import array
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import load_model



# fit a tokenizer
def create_tokenizer(lines):
	tokenizer = Tokenizer()
	tokenizer.fit_on_texts(lines)
	return tokenizer

# calculate the maximum document length
def max_length(lines):
	return max([len(s.split()) for s in lines])

# encode a list of lines
def encode_text(tokenizer, lines, length):
	# integer encode
	encoded = tokenizer.texts_to_sequences(lines)
	# pad encoded sequences
	padded = pad_sequences(encoded, maxlen=length, padding='post')
	return padded

  #for calculating length
def joining(ele):
  return " ".join(str(v) for v in ele)



# create tokenizer
tokenizer = create_tokenizer(X_train)
# calculate max document length
# calculate max document length
joinned_train = X_train.apply(joining)
length = max_length(joinned_train)
# calculate vocabulary size
vocab_size = len(tokenizer.word_index) + 1
print('Max document length: %d' % length)
print('Vocabulary size: %d' % vocab_size)
# encode data
trainX = encode_text(tokenizer,X_train, length)
testX = encode_text(tokenizer, X_test, length)
print(trainX.shape, testX.shape)

#testLabels = list(test_data['sentiment'])
# load the model
model = load_model('model.h5')

# evaluate model on training dataset
loss, acc = model.evaluate([trainX,trainX,trainX], y_train, verbose=0)
print('Train Accuracy: %f' % (acc*100))

# evaluate model on test dataset dataset
loss, acc = model.evaluate([testX,testX,testX], y_test, verbose=0)
print('Test Accuracy: %f' % (acc*100))

Max document length: 23
Vocabulary size: 11562
(11712, 23) (2928, 23)
Train Accuracy: 98.531419
Test Accuracy: 76.229507
